In [14]:
#read all files in dataset. divide them in acted videos and real videos

import os
from datetime import datetime

dataset_root = "/workspace/data/thermix_data/tf_base_dataset/14_tim"
dataset_classes_dirs = [os.path.join(dataset_root, "%d"%i) for i in range(1,6)]

dataset_acted = {}
dataset_real = {}
for videos_dir in dataset_classes_dirs:
    class_id = videos_dir[-1]
    dataset_acted[class_id] = []
    dataset_real[class_id] = []
    videos_names = os.listdir(videos_dir)
    for video_name in videos_names:
        if (not "golden5s" in video_name and not "ipod5" in video_name and not "black5" in video_name) or \
          (datetime.strptime(video_name.split("_")[3],"%Y-%m-%d") <= datetime(2016,6,20)): 
            dataset_acted[class_id].append(video_name)
        else:
            dataset_real[class_id].append(video_name)
            
print "dataset_acted"
print {class_id:len(dataset_acted[class_id]) for class_id in sorted(dataset_acted.keys())}

print "dataset_real"
print {class_id:len(dataset_real[class_id]) for class_id in sorted(dataset_real.keys())}

dataset_acted
{'1': 198, '3': 80, '2': 291, '5': 535, '4': 64}
dataset_real
{'1': 201, '3': 278, '2': 52, '5': 202, '4': 138}


In [15]:
import json

#looks for the person recorded in each video
all_data_root = "/workspace/data/thermix_data/frames_no_movement"

video_owners_backup="/workspace/thermix/ARTraining/video_owners.json"
if not os.path.exists("/workspace/data/thermix_data/frames_no_movement"):
    with open(video_owners_backup) as f:
        video_owners = json.load(f)
else:
    video_owners = {}
    for group in os.listdir(all_data_root):
        if not group[0].isupper():
            continue
        #print group    
        videos_names = os.path.join(all_data_root, group, "14_tim", "3")
        for video_name in os.listdir(videos_names):
            video_owners[video_name] = group
    with open(video_owners_backup,"w") as f:
         json.dump(video_owners, f)

In [16]:
# discard videos that are corrupted.
not_found = []
for class_id in dataset_real.keys():
    videos = dataset_real[class_id]
    for v in videos:
        v_group = video_owners.get(v)
        if not v_group:
            not_found.append((v,class_id))

print "not found:", len(not_found)
for v,c in not_found:
    dataset_real[c].remove(v)
    
groups = {}
for class_id in dataset_real.keys():
    videos = dataset_real[class_id]
    for v in videos:
        v_group = video_owners.get(v).split("_")[0]
            
        if not groups.get(v_group):
            groups[v_group] = [(v,class_id)]
        else:
            groups[v_group].append((v,class_id))
            
print "total_different_people:",len(groups.keys())
#print groups.keys()
print {g:len(groups[g]) for g in groups.keys()}

not found: 42
total_different_people: 11
{u'Julien': 27, u'Luke': 15, u'Charles': 49, u'Victor': 98, u'Marge': 88, u'Henry': 192, u'Fiona': 30, u'Anne': 238, u'Peter': 38, u'Irene': 7, u'Rick': 47}


In [17]:
fold_nr = 1
fold_path="/workspace/data/thermix_data/tf_base_dataset/tf_folds/"
training_h5py_path=os.path.join(fold_path,"%d"%fold_nr, "training.h5")
val_h5py_path=os.path.join(fold_path,"%d"%fold_nr, "validation.h5")

In [19]:
#generate training and validation lists with format 'frame_path class'
import random

def generate_training_validation_dataset(training_proportion=0.8, fold_nr=fold_nr, 
                                         fold_path=fold_path,
                                         dataset_root="/workspace/data/thermix_data/tf_base_dataset/14_tim"):
    training = []
    val = []
    
    for c in dataset_acted.keys():
        training.extend([(v,c) for v in dataset_acted[c]])
        #print len(dataset_acted[c])
        #subset = random.sample(dataset_acted[c], (len(dataset_acted[c])*0.8))
        #training.extend([(v,c) for v in subset])
        #val.extend([(v,c) for v in dataset_acted[c] if v not in subset])
    
    #val = ['Anne',"Luke","Fiona","Irene", 'Julien', 'Victor']
    subset = ["Peter", 'Marge', "Henry", "Rick", 'Charles', ] # random.sample(groups.keys(), int(len(groups.keys())*0.5))
    [training.extend(groups[g]) for g in subset]
    [val.extend(groups[g]) for g in groups.keys() if g not in subset]
    
    print "Training frames distribution"
    count_training = [0,0,0,0,0]
    for v,c in training:
        count_training[int(c)-1] += len(os.listdir(os.path.join(dataset_root,c,v)))
    
    #balancing training set
    min_class_len = min(count_training)
    i = 0
    print max(count_training),min_class_len
    
    frames_training = []
    for v,c in training:
        video_path = os.path.join(dataset_root,c,v)
        frames = os.listdir(video_path)
        for fr in frames:
            frames_training.append((os.path.join(video_path,fr),int(c)-1))
    
    while min_class_len != max(count_training):
        v,c = random.choice(frames_training)
        c_id = c
        if count_training[c_id] > min_class_len:
            frames_training.remove((v,c))
            count_training[c_id] -=1
        else:
            i+=1
            
    print count_training
    
    print "Validation frames distribution"
    count_val = [0,0,0,0,0]
    for v,c in val:
        count_val[int(c)-1] += len(os.listdir(os.path.join(dataset_root,c,v)))
        
    print count_val
    
    frames_validation = []
    for v,c in val:
        video_path = os.path.join(dataset_root,c,v)
        frames = os.listdir(video_path)
        for fr in frames:
            frames_validation.append((os.path.join(video_path,fr),int(c)-1))
    
    training_filepath = os.path.join(fold_path,"%d"%fold_nr, "training_list.txt")
    val_filepath = os.path.join(fold_path,"%d"%fold_nr, "validation_list.txt")
    
    if not os.path.isdir(os.path.dirname(training_filepath)):
        os.makedirs(os.path.dirname(training_filepath))
        
    for fil,dataset in [(training_filepath, frames_training), 
                        (val_filepath,frames_validation)]:
        with open(fil,"w") as f:
            for fr,c in dataset:
                f.write("%s %d\n"%(fr,c))
                    
    return training_filepath, val_filepath
                
training_file,val_file = generate_training_validation_dataset()
#Training frames distribution
#6771 4617
#[4617, 4617, 4617, 4617, 4617]
#Validation frames distribution
#[1554, 510, 1479, 477, 801]

Training frames distribution
6771 4617
[4617, 4617, 4617, 4617, 4617]
Validation frames distribution
[1554, 510, 1479, 477, 801]


In [ ]:
from PIL import Image
from IPython.display import display
import random

with open(training_file) as f:
    lines = f.readlines()

for l in random.sample(lines, 10):
    path = l.split()[0]
    img=Image.open(path)
    print "Category:",l.split()[1]
    display(img)

In [20]:
#save training dataset in h5py format
from training_utils import build_hdf5_thermal_image_dataset

if not os.path.exists(training_h5py_path):
    build_hdf5_thermal_image_dataset(training_file, (224,224), 
                         output_path=training_h5py_path,
                             mode='file', categorical_labels=True,
                             normalize=False, grayscale=True)

In [21]:
#save validation dataset in h5py format
from training_utils import build_hdf5_thermal_image_dataset

if not os.path.exists(val_h5py_path):
    build_hdf5_thermal_image_dataset(val_file, (224,224), 
                         output_path=val_h5py_path,
                             mode='file', categorical_labels=True,
                             normalize=False, grayscale=True)

In [47]:
import tflearn, tensorflow as tf
from tflearn.layers.core import input_data, dropout, fully_connected,reshape
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import batch_normalization#local_response_normalization
#from tflearn.layers.estimator import regression

def model(input_placeholder=None):
    tf_data = input_placeholder or tf.placeholder(tf.float32, shape=(None, 224, 224))
    network = input_data(placeholder=tf_data)
    
    network = reshape(network, [-1,224,224,1])
    
    network = conv_2d(network, 96, 7, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = fully_connected(network, 5, activation='softmax')
    
    return network, tf_data

name = "2016-11-02_190402/checkpoint-1092"
model_dir = "/workspace/data/thermix_data/tf_training"

graph = tf.Graph()
with graph.as_default():
    # Model variables
    #The network to be used
    net, X_ph = model()

    init = tf.initialize_all_variables()
    vars_to_restore = []
    for v in tf.all_variables():
        if "is_training" in v.name:
            print v.name
            continue
        vars_to_restore.append(v)
    saver = tf.train.Saver(vars_to_restore)

import h5py
v_h5f = h5py.File(val_h5py_path)
v_X = v_h5f['X']
v_Y = v_h5f['Y']

# Launch the graph
with tf.Session(graph=graph) as sess:
    tflearn.config.is_training(False,sess)
    sess.run(init)
    saver.restore(sess,os.path.join(model_dir,name))
    
    prediction = []
    ground_truth = []
    i=0
    step = 200
    while len(prediction) <len(v_X):
        res = net.eval({X_ph:v_X[i:i+step]})
        for r in res:
            sorted_indexes = sorted(range(5), key=lambda x:r[x], reverse=True)
            prediction.append(sorted_indexes[0])
        i+=step
    
    for r in v_Y:
        sorted_indexes = sorted(range(5), key=lambda x:r[x], reverse=True)
        ground_truth.append(sorted_indexes[0])

        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ground_truth, prediction)
print cm

BatchNormalization/is_training:0
[[872 134  80 379  89]
 [ 13 416  57  24   0]
 [ 22 354 863 146  94]
 [  7  41  49 343  37]
 [ 60   3   3   3 732]]


In [46]:
import collections, copy

g_err = 0
for jj in range(100):
    a = collections.Counter(ground_truth)
    gt = copy.copy(ground_truth)
    pred = copy.copy(prediction)
    min_class = min(a.values())
    
    while min_class != max(a.values()):
        i = random.choice(range(len(gt)))
        if a[gt[i]] > min_class:
            a[gt[i]] -= 1
            gt.pop(i)
            pred.pop(i)
    err = 0
    for x in range(len(gt)):
        err += 1 if gt[x] != pred[x] else 0
    #print float(err)/len(gt)
    g_err += float(err)/len(gt)
print g_err/100

0.280708595388


In [49]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='pusiol', api_key='m6iurl7f89')

def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

def plotly_confusion_matrix(cm, title):
    classes=["lying", "sitting","standing", "people", "background"]
    annotations = []
    for n, row in enumerate(cm):
        for m, val in enumerate(row):
            var = cm[n][m]
            annotations.append(
                dict(
                    text=str(val),
                    x=classes[m], y=classes[n],
                    xref='x1', yref='y1',
                    font=dict(color='white'),
                    showarrow=False)
                )
    data = [
        go.Heatmap(
            x=classes,
            y=classes,
            z=cm,
#            colorscale='Viridis'
        )
    ]
    
    layout = go.Layout(
            title='Confusion Matrix',
            annotations=annotations,
            xaxis=dict(title='Predicted value',),
            yaxis=dict(title='Real value',)
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename="confusion_matrix_test")
    
plotly_confusion_matrix(cm, title=name)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.

